### Pair Programming ETL Transformación II

En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [42]:
import pandas as pd
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")

In [33]:
class Energy:
    
    def __init__(self, start_year, end_year): 
        """Método constructor, recibe los siguientes parámetros:
            - start_year: año de inicio de la consulta
            - end_year: año fin de la consulta"""
        
        self.start_year = start_year
        self.end_year = end_year
        
    def energy_spain(self): 
        """Recibe los parámetros del método constructor. Llama a API Red Eléctrica de España
            Output: devuelve un dataframe con el consumo, fecha y tipo de energía consumida para todos los años consultados (renovable/no renovable) por cada día total de España"""

        df_spain = pd.DataFrame() # Creamos un dataframe vacío en el que se vayan uniendo nuestros resultados. 

        
        for year in range(self.start_year, self.end_year):

            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=day"
            
            response = requests.get(url)

            print(f"Año {year}: {response.status_code}")

            for i in range(len(pd.json_normalize(response.json()["included"]))):

                df = pd.json_normalize(response.json()["included"][i]["attributes"]["values"]) #Creamos un dataframe únicamente con los valores que nos interesan, que son "value", "percentage",
                # y "datetime"

                df["energy_type"] = response.json()["included"][i]["attributes"]["type"] # Creamos nueva columna con "tipo_energia", dejando fuera lo que no nos interesaba.

                df_spain = pd.concat([df_spain, df], axis = 0) # Hacemos un concat para que cada for se vaya añadiendo al DataFrame vacío creado anteriormente.
                
        return df_spain
    
    def energy_location(self):
        """Recibe los parámetros del método constructor. Llama a API Red Eléctrica de España
            Output: devuelve un dataframe con el consumo, fecha y tipo de energía consumida para todos los años consultados (renovable/no renovable) por cada año dividido por comunidades"""
        

        cod_location = {"Ceuta": 8744, "Melilla": 8745, "Andalucía": 4, "Aragón": 5, "Cantabria": 6, "Castilla - La Mancha": 7, "Castilla y León": 8, "Cataluña": 9, "País Vasco": 10,
                        "Principado de Asturias": 11, "Comunidad de Madrid": 13, "Comunidad Foral de Navarra": 14, "Comunitat Valenciana": 15, "Extremadura": 16, "Galicia": 17,
                        "Illes Balears": 8743, "Canarias": 8742, "Región de Murcia": 21, "La Rioja": 20}

        df_location = pd.DataFrame() #Creamos un dataframe vacío en el que se vayan uniendo nuestros resultados. 

        
        for year in range(self.start_year, self.end_year):

            for key, value in cod_location.items(): 

                url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={value}"
                
                response = requests.get(url)

                print(f"Año {year}, Comunidad {key}: {response.status_code}")

                for i in range(len(pd.json_normalize(response.json()["included"]))):

                    df2 = pd.json_normalize(response.json()["included"][i]["attributes"]["values"]) #Creamos un dataframe únicamente con los valores que nos interesan, que son "value", "percentage",
                    # y "datetime"

                    df2["energy_type"] = response.json()["included"][i]["attributes"]["type"]

                    df2["location"] = key

                    df2["id_location"] = key #Cremos columnas con otros elementos que nos interesaban de la API, dejando fuera lo que no nos interesaba.

                    df_location = pd.concat([df_location, df2], axis = 0) # Hacemos un concat para que cada for se vaya añadiendo al DataFrame vacío creado anteriormente.
                
        return df_location
    
    def clean(self, dataframe):
        """Recibe el siguiente parámetro:
            - dataframe: nombre del dataframe que se quiere limpiar 
            Output: devuelve el dataframe con:
                - las columnas "percentage" y "value" redondeadas a 2 decimales
                - la fecha en formato YYYY-MM-DD y tipo datetime
                - eliminando la columna inicial "datetime"""
        
        dataframe[["percentage", "value"]] = dataframe[["percentage", "value"]].apply(lambda element : element.round(2))
        
        dataframe["date"] = dataframe["datetime"].str.split("T", n = 1, expand = True).get(0).astype("datetime64[ns]")
        
        dataframe.drop("datetime", axis= 1, inplace= True)
        
        return dataframe
    
    
    def export(self, dataframe, path_name, format= "pkl"):
        """Recibe los siguientes parámetros:
            - dataframe: dataframe a guardar
            - path_name: ruta y nombre del archivo que se quiere guardar
            - format: formato en el que se quiere guardar, puede ser: pkl, csv o excel. Por defecto exporta a pkl.
            Output: dataframe exportado"""
    
        if format == "pkl":
            
            return dataframe.to_pickle(f"{path_name}.pkl")
        
        elif format == "csv":
            
            return dataframe.to_csv(f"{path_name}.csv")
            
        elif format == "excel":
            
            return dataframe.to_excel(f"{path_name}.xlsx")
        else:
            print("Formato incorrecto, selecciona 'pkl', 'csv' o 'excel'")

In [34]:
df = Energy(2020, 2022)

In [35]:
df_spain = df.energy_spain()
df_spain.head(2)

Año 2020: 200
Año 2021: 200


,value,percentage,datetime,energy_type
0,213821.4245,0.359071,2020-01-01T00:00:00.000+01:00,Renovable
1,270043.7695,0.381460,2020-01-02T00:00:00.000+01:00,Renovable


In [36]:
df_location = df.energy_location()
df_location.head(2)

Año 2020, Comunidad Ceuta: 200
Año 2020, Comunidad Melilla: 200
Año 2020, Comunidad Andalucía: 200
Año 2020, Comunidad Aragón: 200
Año 2020, Comunidad Cantabria: 200
Año 2020, Comunidad Castilla - La Mancha: 200
Año 2020, Comunidad Castilla y León: 200
Año 2020, Comunidad Cataluña: 200
Año 2020, Comunidad País Vasco: 200
Año 2020, Comunidad Principado de Asturias: 200
Año 2020, Comunidad Comunidad de Madrid: 200
Año 2020, Comunidad Comunidad Foral de Navarra: 200
Año 2020, Comunidad Comunitat Valenciana: 200
Año 2020, Comunidad Extremadura: 200
Año 2020, Comunidad Galicia: 200
Año 2020, Comunidad Illes Balears: 200
Año 2020, Comunidad Canarias: 200
Año 2020, Comunidad Región de Murcia: 200
Año 2020, Comunidad La Rioja: 200
Año 2021, Comunidad Ceuta: 200
Año 2021, Comunidad Melilla: 200
Año 2021, Comunidad Andalucía: 200
Año 2021, Comunidad Aragón: 200
Año 2021, Comunidad Cantabria: 200
Año 2021, Comunidad Castilla - La Mancha: 200
Año 2021, Comunidad Castilla y León: 200
Año 2021, Comu

,value,percentage,datetime,energy_type,location,id_location
0,199198.1020,1.000000,2020-01-01T00:00:00.000+01:00,No renovable,Ceuta,Ceuta
0,5602.2625,0.026933,2020-01-01T00:00:00.000+01:00,Renovable,Melilla,Melilla


In [37]:
df_spain = df.clean(df_spain)
df_spain.head(2)

,value,percentage,energy_type,date
0,213821.42,0.36,Renovable,2020-01-01
1,270043.77,0.38,Renovable,2020-01-02


In [38]:
df_spain.head(2)

,value,percentage,energy_type,date
0,213821.42,0.36,Renovable,2020-01-01
1,270043.77,0.38,Renovable,2020-01-02


In [39]:
df_location = df.clean(df_location)
df_location.head(2)

,value,percentage,energy_type,location,id_location,date
0,199198.10,1.00,No renovable,Ceuta,Ceuta,2020-01-01
0,5602.26,0.03,Renovable,Melilla,Melilla,2020-01-01


In [40]:
df.export(df_spain, "data/energias_años", "csv")

In [41]:
df.export(df_location, "data/energias_comunidades")